In [25]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize  import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
from ast import literal_eval

[nltk_data] Downloading package punkt to /Users/slash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/slash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/slash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/slash/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
#  copy this link to download dataset
'''

https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe

'''

'\n\nhttps://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe\n\n'

In [26]:
data = pd.read_csv("Hotel_Reviews.csv")
data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [27]:
data.shape

(515738, 17)

In [28]:
# for simplicty will change the countroes name to Alpgha-2 codes
data.Hotel_Address = data.Hotel_Address.str.replace("Netherlands", "NL")
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")
data.Hotel_Address = data.Hotel_Address.str.replace("France", "FR")
data.Hotel_Address = data.Hotel_Address.str.replace("Italy", "IT")
data.Hotel_Address = data.Hotel_Address.str.replace("Spain", "SP")
data.Hotel_Address = data.Hotel_Address.str.replace("Austria", "AT")

In [29]:
data.Hotel_Address.head(5)

0     s Gravesandestraat 55 Oost 1092 AA Amsterdam NL
1     s Gravesandestraat 55 Oost 1092 AA Amsterdam NL
2     s Gravesandestraat 55 Oost 1092 AA Amsterdam NL
3     s Gravesandestraat 55 Oost 1092 AA Amsterdam NL
4     s Gravesandestraat 55 Oost 1092 AA Amsterdam NL
Name: Hotel_Address, dtype: object

In [30]:
#  will add or deriv a new cloumn from last string at hotel address with specify where the hotel loctated
# data["countries"] =[ x[-2:] for x in data.Hotel_Address]
data["countries"] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])

# to make sure that Counties added will use unique() to retrieve unique words at the column
data.countries.unique()

# data[data['countries'] == 'NL']

array(['NL', 'UK', 'FR', 'SP', 'IT', 'AT'], dtype=object)

In [31]:
# will work in new dataset
data.drop(['Additional_Number_of_Scoring', 'Review_Date',
       'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'],axis=1,inplace=True)

data

,Hotel_Address,Average_Score,Hotel_Name,Tags,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",NL
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",NL
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",NL
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",NL
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam NL,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",NL
...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with older childre...",AT
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with young childre...",AT
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Family with older childre...",AT
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,8.1,Atlantis Hotel Vienna,"[' Leisure trip ', ' Group ', ' Standard Tripl...",AT


In [32]:
# create a function help us to convert object(pandas.core.series.Series) to List then to Sring from Tags
def convert(column):
    column = column[0]
    if (type(column) != list):
        return "".join(literal_eval(column))
    else:
        return column
# dataset.head(1)

In [33]:
# will call the convert function to change object dataType of column Tags to Srting
data['Tags'] = data[['Tags']].apply(convert, axis= 1)
data.Tags.head(3)

/var/folders/bb/lk1lsv4s6r3f8nb7k99hxq6w0000gn/T/ipykernel_33763/2726750827.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  column = column[0]


0     Leisure trip  Couple  Duplex Double Room  Sta...
1     Leisure trip  Couple  Duplex Double Room  Sta...
2     Leisure trip  Family with young children  Dup...
Name: Tags, dtype: object

In [34]:
# To simplicty => change Countries and Tags columns to lowercase
data["Tags"]= data["Tags"].str.lower()
data["countries"]= data["countries"].str.lower()

In [36]:
#  model
# location => Counties , description => Tags
def recomender(location , description):
#  lemmatiaztion for Tags column
    description =  description.lower()
    word_tokenize(description)
    stop_words  =  stopwords.words("english")
    lemm  =  WordNetLemmatizer()

#     cheching there is a stopwords, if no will add at list
    filtered= {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))


#    creating varible refer to all countires == to location i want
    country = data[data['countries']==location.lower()]
#     set index by using numpy                   shape[0]=> [0]> num of rows
    country = country.set_index(np.arange(country.shape[0]))

    list1 = []
    list2 = []
    cos   = []

#     for loop to iteration of all rows
    for i in range(country.shape[0]):

#     make tokinze and then, filter from stopwords
        temp_token = word_tokenize(country["Tags"][i])
        temp_set   = [word for word in temp_token if not word in stop_words]
#       list grouping the lemmatizetion of words tokenization
        temp2_set =set()
        for xx in temp_set:
            temp2_set.add(lemm.lemmatize(xx))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    country['similarity'] = cos
#     arange hotels by using tags rating
    country = country.sort_values(by="similarity", ascending=False)
#                                   require            ,optional ^defult
    country.drop_duplicates(subset='Hotel_Name', keep="first", inplace= True)

    country.sort_values('Average_Score', ascending=False, inplace=True)
    country.reset_index(inplace = True)
    return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head(10)

In [37]:
recomender('FR','Business trip')

,Hotel_Name,Average_Score,Hotel_Address
0,Ritz Paris,9.8,15 Place Vend me 1st arr 75001 Paris FR
1,H tel de La Tamise Esprit de France,9.6,4 rue d Alger 1st arr 75001 Paris FR
2,Le Narcisse Blanc Spa,9.5,19 Boulevard De La Tour Maubourg 7th arr 75007...
3,Hotel The Peninsula Paris,9.5,19 avenue Kleber 16th arr 75116 Paris FR
4,La Chambre du Marais,9.4,85 87 RUE DES ARCHIVES 3rd arr 75003 Paris FR
5,Goralska R sidences H tel Paris Bastille,9.4,7 Boulevard Bourdon 4th arr 75004 Paris FR
6,Nolinski Paris,9.4,16 Avenue de l Opera 1st arr 75001 Paris FR
7,Splendide Royal Paris,9.4,18 Rue du Cirque 8th arr 75008 Paris FR
8,Hotel Monge,9.4,55 rue Monge 5th arr 75005 Paris FR
9,Hotel Eiffel Blomet,9.4,78 Rue Blomet 15th arr 75015 Paris FR
